In [24]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [25]:
#load data and ensure it's good
df = pd.read_csv("./Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")

df.columns = df.columns.str.lstrip()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/s', '_s')
df.columns = df.columns.str.replace('Fwd_Header_Length.1', 'Fwd_Header_Length1')
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df=df.fillna(0)
df["Target"] = np.where(df["Label"] == "BENIGN", 0, 1)

In [26]:
df.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Min,Bwd_Packet_Length_Mean,Bwd_Packet_Length_Std,Flow_Bytes_s,Flow_Packets_s,Flow_IAT_Mean,Flow_IAT_Std,Flow_IAT_Max,Flow_IAT_Min,Fwd_IAT_Total,Fwd_IAT_Mean,Fwd_IAT_Std,Fwd_IAT_Max,Fwd_IAT_Min,Bwd_IAT_Total,Bwd_IAT_Mean,Bwd_IAT_Std,Bwd_IAT_Max,Bwd_IAT_Min,Fwd_PSH_Flags,Bwd_PSH_Flags,Fwd_URG_Flags,Bwd_URG_Flags,Fwd_Header_Length,Bwd_Header_Length,Fwd_Packets_s,Bwd_Packets_s,Min_Packet_Length,Max_Packet_Length,Packet_Length_Mean,Packet_Length_Std,Packet_Length_Variance,FIN_Flag_Count,SYN_Flag_Count,RST_Flag_Count,PSH_Flag_Count,ACK_Flag_Count,URG_Flag_Count,CWE_Flag_Count,ECE_Flag_Count,Down/Up_Ratio,Average_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Fwd_Header_Length1,Fwd_Avg_Bytes/Bulk,Fwd_Avg_Packets/Bulk,Fwd_Avg_Bulk_Rate,Bwd_Avg_Bytes/Bulk,Bwd_Avg_Packets/Bulk,Bwd_Avg_Bulk_Rate,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Target
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN,0


In [27]:
print(df.shape)
print(df.isna().sum())
print()
print(df.isin([np.inf, -np.inf]).sum())

(225745, 80)
Destination_Port               0
Flow_Duration                  0
Total_Fwd_Packets              0
Total_Backward_Packets         0
Total_Length_of_Fwd_Packets    0
Total_Length_of_Bwd_Packets    0
Fwd_Packet_Length_Max          0
Fwd_Packet_Length_Min          0
Fwd_Packet_Length_Mean         0
Fwd_Packet_Length_Std          0
Bwd_Packet_Length_Max          0
Bwd_Packet_Length_Min          0
Bwd_Packet_Length_Mean         0
Bwd_Packet_Length_Std          0
Flow_Bytes_s                   0
Flow_Packets_s                 0
Flow_IAT_Mean                  0
Flow_IAT_Std                   0
Flow_IAT_Max                   0
Flow_IAT_Min                   0
Fwd_IAT_Total                  0
Fwd_IAT_Mean                   0
Fwd_IAT_Std                    0
Fwd_IAT_Max                    0
Fwd_IAT_Min                    0
Bwd_IAT_Total                  0
Bwd_IAT_Mean                   0
Bwd_IAT_Std                    0
Bwd_IAT_Max                    0
Bwd_IAT_Min                   

In [28]:
features = np.float32(df.loc[:, ~df.columns.isin(['Flow_ID', 'Source_IP', 'Destination_IP', 'Timestamp', 'Label', 'Target'])].copy().values) #reduce memory usage
label = df.loc[:, df.columns.isin(['Target'])].copy().values.ravel()

In [29]:
from sklearn.ensemble import RandomForestClassifier #use this when label is categorical

# Train a random forest classifier on the dataset
forest = RandomForestClassifier(n_estimators = 100, random_state=42)
forest.fit(features, label)

# Get the feature importances and sort them in descending order
importances = forest.feature_importances_
sorted_indices = importances.argsort()[::-1]
feature_names = df.loc[:, ~df.columns.isin(['Flow_ID', 'Source_IP', 'Destination_IP', 'Timestamp', 'Label', 'Target'])].copy().columns

# Print the top 10 features by importance
top_features = 10
for index in sorted_indices[:top_features]:
    print(f"{feature_names[index]}: {importances[index]}")

KeyboardInterrupt: 

In [40]:
import plotly.graph_objs as go
import json
import plotly as plotly

# Create a horizontal bar plot of feature importances
trace = go.Bar(
    y=feature_names[sorted_indices][:top_features][::-1],
    x=importances[sorted_indices][:top_features][::-1],
    orientation='h',
    marker=dict(color=['#ff7f0e','#1f77b4','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']),
)

# Add labels to the plot
layout = go.Layout(
    xaxis=dict(title='Importance'),
    yaxis=dict(title='Features'),
    title='Top 10 Features by Importance',
    margin=dict(l=150, r=50, b=100, t=100),
)

# Add a frame around each bar and the importance value to the middle of each bar
annotations = []
for i in range(top_features):
    annotations.append(dict(xref='x', yref='y',
                            x=importances[sorted_indices][:top_features][::-1][i], y=feature_names[sorted_indices][:top_features][::-1][i],
                            text=str(round(importances[sorted_indices][:top_features][::-1][i], 3)),
                            font=dict(family='Arial', size=12, color='white'),
                            showarrow=False,
                            align='center',
                           ))
    
    trace.marker.line.width = 1
    trace.marker.line.color = '#000000'
    
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(annotations=annotations)
fig.show()

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('FeatureSelection.json', 'w') as f:
    f.write(fig_json)

In [ ]:
names = ['Destination_Port', 'Fwd_Packet_Length_Max', "Avg_Fwd_Segment_Size", "Fwd_Packet_Length_Mean", "Subflow_Fwd_Bytes",
        "Total_Length_of_Fwd_Packets", "Source_Port", "Init_Win_bytes_forward"]
'''"Fwd_Packet_Length_Std", "Init_Win_bytes_backward",
        "act_data_pkt_fwd", "Bwd_Packet_Length_Mean", "Avg_Bwd_Segment_Size", "Fwd_IAT_Total", "min_seg_size_forward", "Fwd_IAT_Max",
        "Bwd
        _Packet_Length_Max", "Average_Packet_Size", "Bwd_Packet_Length_Std", "Packet_Length_Mean"'''
features = np.float32(df.loc[:, df.columns.isin(names)].copy().values) #reduce memory usage

In [30]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import time

# Use the top 10 features
X = features
y = df['Target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate several classifiers
models = {
    "Gaussian Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    s_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    e_time = time.time()
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"{name}:")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 score: {f1:.3f}")
    print(f"Running time: {e_time - s_time:.3f} seconds\n")

Gaussian Naive Bayes:
Accuracy: 0.808
Precision: 0.749
Recall: 0.999
F1 score: 0.856
Running time: 0.263 seconds



KeyboardInterrupt: 

In [36]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve
import plotly as plotly

# Raandom Forest

# Define the model to use
model = RandomForestClassifier()

X = np.float32(df.loc[:, df.columns.isin(['Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Max', "act_data_pkt_fwd", "Avg_Fwd_Segment_Size", "Fwd_IAT_Std",
        "Subflow_Fwd_Bytes", "Total_Fwd_Packets", "Init_Win_bytes_forward", "Fwd_Header_Length1", "Destination_Port"])].copy().values)
y = df['Target'].values

# Compute the learning curve for the model
train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, n_jobs=-1,
                                                            train_sizes=np.linspace(0.1, 1.0, 10))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Create a new plot
fig = go.Figure()

# Add the training and test scores to the plot
fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines+markers', name="Training score", line=dict(color="red")))
fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines+markers', name="Testing score", line=dict(color="green")))

# Update the layout of the plot
fig.update_layout(title='Random Forest Learning Curve', xaxis_title='Training Set Size', yaxis_title='Score')

# Show the interactive plot

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('RandomForest.json', 'w') as f:
    f.write(fig_json)
import json

In [37]:
from sklearn.naive_bayes import GaussianNB

# Gaussian Naive Bayes

# Define the model to use
model = GaussianNB()

X = np.float32(df.loc[:, df.columns.isin(['Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Max', "act_data_pkt_fwd", "Avg_Fwd_Segment_Size", "Fwd_IAT_Std",
        "Subflow_Fwd_Bytes", "Total_Fwd_Packets", "Init_Win_bytes_forward", "Fwd_Header_Length1", "Destination_Port"])].copy().values)
y = df['Target'].values

# Compute the learning curve for the model
train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, n_jobs=-1,
                                                            train_sizes=np.linspace(0.1, 1.0, 10))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Create a new plot
fig = go.Figure()

# Add the training and test scores to the plot
fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines+markers', name="Training score", line=dict(color="red")))
fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines+markers', name="Testing score", line=dict(color="green")))

# Update the layout of the plot
fig.update_layout(title='Random Forest Learning Curve', xaxis_title='Training Set Size', yaxis_title='Score')

# Show the interactive plot

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('GaussianNB.json', 'w') as f:
    f.write(fig_json)
import json

In [38]:
from sklearn.neighbors import KNeighborsClassifier

# KNeighborsClassifier

# Define the model to use
model = KNeighborsClassifier()

X = np.float32(df.loc[:, df.columns.isin(['Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Max', "act_data_pkt_fwd", "Avg_Fwd_Segment_Size", "Fwd_IAT_Std",
        "Subflow_Fwd_Bytes", "Total_Fwd_Packets", "Init_Win_bytes_forward", "Fwd_Header_Length1", "Destination_Port"])].copy().values)
y = df['Target'].values

# Compute the learning curve for the model
train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, n_jobs=-1,
                                                            train_sizes=np.linspace(0.1, 1.0, 10))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Create a new plot
fig = go.Figure()

# Add the training and test scores to the plot
fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines+markers', name="Training score", line=dict(color="red")))
fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines+markers', name="Testing score", line=dict(color="green")))

# Update the layout of the plot
fig.update_layout(title='Random Forest Learning Curve', xaxis_title='Training Set Size', yaxis_title='Score')

# Show the interactive plot

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('KNeighbors.json', 'w') as f:
    f.write(fig_json)


In [39]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree Classifier

# Define the model to use
model = DecisionTreeClassifier()

X = np.float32(df.loc[:, df.columns.isin(['Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Max', "act_data_pkt_fwd", "Avg_Fwd_Segment_Size", "Fwd_IAT_Std",
        "Subflow_Fwd_Bytes", "Total_Fwd_Packets", "Init_Win_bytes_forward", "Fwd_Header_Length1", "Destination_Port"])].copy().values)
y = df['Target'].values

# Compute the learning curve for the model
train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5, n_jobs=-1,
                                                            train_sizes=np.linspace(0.1, 1.0, 10))
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Create a new plot
fig = go.Figure()

# Add the training and test scores to the plot
fig.add_trace(go.Scatter(x=train_sizes, y=train_mean, mode='lines+markers', name="Training score", line=dict(color="red")))
fig.add_trace(go.Scatter(x=train_sizes, y=test_mean, mode='lines+markers', name="Testing score", line=dict(color="green")))

# Update the layout of the plot
fig.update_layout(title='Random Forest Learning Curve', xaxis_title='Training Set Size', yaxis_title='Score')

# Show the interactive plot

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('DecisionTree.json', 'w') as f:
    f.write(fig_json)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# LASSO Regression for Feature Selection

X = np.float32(df.loc[:, ~df.columns.isin(['Flow_ID', 'Source_IP', 'Destination_IP', 'Timestamp', 'Label', 'Target'])].copy().values)
y = df['Target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Testing multiple L1 penalty

l1_lambdas = np.logspace(1, 7, 7, base=10)

lasso_data = []

for reg_coef in l1_lambdas:
  l1_penalty = reg_coef
  lasso_model = Lasso(alpha=reg_coef)
  model = lasso_model.fit(X_train, y_train)
  lasso_data.append({
    'l1_penalty' : l1_penalty,
    'model' : lasso_model,
    'rmse_train' : mean_squared_error(y_train, model.predict(X_train), squared=False),
  })
lasso_data = pd.DataFrame(lasso_data)

lasso_data

c:\Users\jonth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.446e+03, tolerance: 4.434e+00

c:\Users\jonth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.184e+03, tolerance: 4.434e+00

c:\Users\jonth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.543e+03,

,l1_penalty,model,rmse_train
0,10.0,Lasso(alpha=10.0),0.273985
1,100.0,Lasso(alpha=100.0),0.303884
2,1000.0,Lasso(alpha=1000.0),0.330290
3,10000.0,Lasso(alpha=10000.0),0.365711
4,100000.0,Lasso(alpha=100000.0),0.401447
5,1000000.0,Lasso(alpha=1000000.0),0.454019
6,10000000.0,Lasso(alpha=10000000.0),0.495505


In [ ]:
# Finding best L1 penalty

best_l1 = lasso_data.loc[lasso_data["rmse_train"].idxmin()]["l1_penalty"]

lasso_model = Lasso(alpha=best_l1)
model = lasso_model.fit(X_train, y_train)

rmse_test_lasso = mean_squared_error(y_test, model.predict(X_test), squared=False)

num = 0
for i in lasso_model.coef_:
  if i == 0:
    num += 1

num_zero_coeffs_lasso = num

print(lasso_model.coef_)

print('Best L1 Penalty', best_l1)
print('Test RMSE', rmse_test_lasso)
print('Num Zero Coeffs', num_zero_coeffs_lasso)

[-6.0234042e-06 -2.0992474e-08  0.0000000e+00 -0.0000000e+00
 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00
 -0.0000000e+00 -0.0000000e+00  3.7929298e-05 -0.0000000e+00
  8.5251442e-05  0.0000000e+00 -4.1942785e-10 -6.2935023e-08
  3.4461448e-07 -3.3618544e-08 -5.5816338e-09 -2.8750014e-07
  1.7231226e-08 -1.0429339e-07  6.1614053e-08  4.5540278e-09
  5.0699342e-08 -2.2343956e-10 -5.1735544e-08 -3.2847392e-08
  2.0949368e-08  9.5777333e-09 -0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00 -0.0000000e+00 -0.0000000e+00
  0.0000000e+00 -5.1940992e-07 -0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00 -1.6430898e-08 -0.0000000e+00
 -0.0000000e+00 -0.0000000e+00  0.0000000e+00  0.0000000e+00
 -0.0000000e+00  0.0000000e+00 -0.0000000e+00 -0.0000000e+00
  2.2420030e-04 -0.0000000e+00  0.0000000e+00 -0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00 -8.6537711e-06
 -0.0000000e+00 -2.15071

c:\Users\jonth\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.446e+03, tolerance: 4.434e+00



In [ ]:
# Print Features with non-zero coefficients

best_model_lasso = model

zero_coef_features = []
nonzero_coef_features = []

best_coef_features = []

for feature, coef in zip(df.columns, best_model_lasso.coef_):
  if abs(coef) <= 10 ** -17:
    zero_coef_features.append(feature)
  else:
    nonzero_coef_features.append(feature)
    best_coef_features.append({
      'feature' : feature,
      'coef' : abs(coef)
    })

best_coef_features = pd.DataFrame(best_coef_features)

print("Features with coefficient == 0:", zero_coef_features)
print("Features with coefficient != 0:", nonzero_coef_features)

Features with coefficient == 0: ['Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Std', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length', 'Bwd_Header_Length', 'Fwd_Packets_s', 'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Std', 'FIN_Flag_Count', 'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count', 'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Flag_Count', 'Down/Up_Ratio', 'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size', 'Fwd_Header_Length1', 'Fwd_Avg_Bytes/Bulk', 'Fwd_Avg_Packets/Bulk', 'Fwd_Avg_Bulk_Rate', 'Bwd_Avg_Bytes/Bulk', 'Bwd_Avg_Packets/Bulk', 'Bwd_Avg_Bulk_Rate', 'Subflow_Fwd_Packets', 'Subflow_Bwd_Packets', 'act_data_pkt_fwd', 'min_seg_size_forward']
Features with coefficient != 0: ['Destin

In [ ]:
sorted = best_coef_features.sort_values('coef', ascending=False)

top_ten = sorted.head(10)




In [43]:
trace = go.Bar(
    y=top_ten['feature'],
    x=top_ten['coef'],
    orientation='h',
    marker=dict(color=['#ff7f0e','#1f77b4','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']),
)

x_min = np.min(top_ten['coef']) 
x_max = np.max(top_ten['coef'])

# Add labels to the plot
layout = go.Layout(
    xaxis=dict(title='Coefficient', range=[x_min, x_max + 0.0001]),
    yaxis=dict(title='Features'),
    title='Top 10 Features by Coefficients with LASSO Regression',
)
    
fig = go.Figure(data=[trace], layout=layout)
fig.update_layout(annotations=annotations)
fig.show()

fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
with open('FeatureSelection2.json', 'w') as f:
    f.write(fig_json)